In [402]:
import folium
import pickle
import simplejson
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np

# Make the dataframe

In [753]:
month = 2 #Oct
day_of_week = 1 #Wed
hour = 1 #From 3pm to 4pm

f=open('network_mesh_space=0.0025_month='+str(month)+'_day='+str(day_of_week)+'.pickle','rb')
network = pickle.load(f)
f.close()

In [754]:
trips = {} #{center_id: [outgoing trips, incoming trips]}

count_pick_ups = network.loc(axis=0)[hour,:,:,:]
#count_pick_ups = network.loc[(hour,slice(None),slice(None),slice(None)),:]
count_pick_ups_index = count_pick_ups.index.values
for i in count_pick_ups_index:
    try:
        trips[str(int(i[2]))][0] += int(count_pick_ups.ix[i])
    except KeyError:
        trips[str(int(i[2]))] = [int(count_pick_ups.ix[i]),0]

count_drop_offs = network.loc(axis=0)[:,hour,:,:]
count_drop_offs_index = count_drop_offs.index.values
for i in count_drop_offs_index:
    try:
        trips[str(int(i[3]))][1] += int(count_drop_offs.ix[i])
    except KeyError:
        trips[str(int(i[3]))] = [0,int(count_drop_offs.ix[i])]


In [755]:
tk = trips.keys()
tv = trips.values()

map_data = pd.DataFrame({'centers': tk, 'activity' : [i[0]+i[1] for i in tv], 
                         'attractiveness': [i[1]-i[0] for i in tv]})

In [756]:
map_data.head()

,activity,attractiveness,centers
0,93,11,23990
1,23,5,23991
2,3,-1,22469
3,2,-2,23993
4,1,1,22465


In [752]:
map_data.head()

,activity,attractiveness,centers
0,1,-1,23994
1,350,-72,23990
2,157,31,23991
3,4,2,22469
4,4,2,23993


In [342]:
map_data.activity.max()

160

# Make the grid

In [507]:
f=open('centers_long_lat_id_mesh_space=0.0025.pickle','rb')
grid = pickle.load(f)
f.close()

In [508]:
grid_json = {"type":"FeatureCollection", "features":[]}

In [509]:
add_mesh = 0.0025/2
for index, row in grid.iterrows():
    if str(int(row[2])) in trips.keys():
        popup_content= "Incoming: "+str(trips[str(int(row[2]))][0])+"<br /> outgoing: "+str(trips[str(int(row[2]))][1]) +"<br /> Activity: "+str(trips[str(int(row[2]))][1] + trips[str(int(row[2]))][0]) +"<br /> Attractiveness: "+str(trips[str(int(row[2]))][1] - trips[str(int(row[2]))][0])
        coord = [[row[0]+add_mesh,row[1]+add_mesh],[row[0]+add_mesh,row[1]-add_mesh],[row[0]-add_mesh,row[1]-add_mesh],[row[0]-add_mesh,row[1]+add_mesh],[row[0]+add_mesh,row[1]+add_mesh]]
        dd = {"type":"Feature","id":str(int(row[2])),
              "properties":{"name":str(int(row[2])),"popupContent":popup_content},
             "geometry":{"type":"Polygon","coordinates":[coord]}
             }
        grid_json['features'].append(dd)

In [512]:
with open('grid.json', 'w') as outfile:
    simplejson.dump(grid_json, outfile)

In [513]:
grid_json = simplejson.dumps(grid_json)

# Make the map

In [746]:
import folium

map_1 = folium.Map(location=[40.74006, -73.98605], zoom_start=13, tiles='Stamen Terrain')
map_1.lat_lng_popover()

In [743]:
map_1.geo_json(geo_str=grid_json, data=map_data,
               columns=['centers', 'activity'],
               data_out='data.json',
               threshold_scale=[0, 250, 500, 750, 1000, 2000],
               key_on='feature.id',fill_color='BuPu', fill_opacity=0.5, line_weight=1,
               line_opacity=0.8,line_color='black',
               legend_name='Activity Rate',reset=True)

In [747]:
map_1.circle_marker(location=[40.74006, -73.98605],radius=60,popup='hi',line_color='red',
                  fill_color='red', fill_opacity=0.4)

In [748]:
map_1.create_map(path='nyc.html')